## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def clean(engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除
    """
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")

In [9]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [10]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

In [11]:
def exe_df(stmt:str,engine:sqlalchemy.engine=engine)->pd.DataFrame:
    """
    sql(select)を実行してdataframeを返す
    """
    with engine.connect() as conn:
        df=pd.read_sql(stmt,conn)
    return df 

## List

In [12]:
#10-1
clean()
exe("""
create table 家計簿(
日付 date,
費目ID integer
)
""")

In [13]:
#10-3
clean()
exe("""
create table 家計簿(
費目ID integer,
メモ varchar(100) default '不明',
入金額 integer default 0
)
""")

In [14]:
#10-4
exe("""
alter table 家計簿 add 関連日 date
""")

In [15]:
#10-5
clean()
exe("""
create table 家計簿(
メモ varchar(100) default '不明' not null,
入金額 integer default 0 check(入金額>=0)
)
""")
exe("""
create table 費目(
ID integer,
名前 varchar(40) unique
)
""")

In [16]:
#10-7
#複合主キーの指定が可能
clean()
exe("""
create table 費目(
ID integer,
名前 varchar(40) unique,
primary key(ID, 名前)
)
""")

In [17]:
#10-6
clean()
exe("""
create table 費目(
ID integer primary key,
名前 varchar(40) unique
)
""")
exe("insert into 費目  (ID, 名前) values (1,'name1')")
exe("insert into 費目  (ID, 名前) values (2,'name2')")
exe_df("select * from 費目")

,id,名前
0,1,name1
1,2,name2


In [18]:
#10-10
exe("""
create table 家計簿(
日付 date not null,
費用ID integer references 費目(ID)
)
""")

## Q

In [19]:
#10-2
clean()
exe("""
create table 学部(
ID char(1) primary key,
名前 varchar(20) unique not null,
備考 varchar(100) default '特になし' not null
)
""")

In [20]:
#10-3
exe("""
create table 学生(
学籍番号 char(8) primary key,
名前 varchar(30) not null,
生年月日 date not null,
学部ID char(1) references 学部(ID)
)
""")